# Baseline MLP Classification Experiment

In [1]:
import pandas as pd

## Data Handling

In [2]:
# load data
tag_metal = "../../my_data/identification-dataset/nlos/anTag/metal/data1/metal-anTag-data1_data.csv"
df_tag = pd.read_csv(tag_metal)
df_tag.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/identification-dataset/nlos/anTag/metal/data1/metal-anTag-data1_data.csv'

In [ ]:
# splitting in anchor information
df_an1 = df_tag[['tdoa12', 'snr_an1', 'power_dif_an1', 'an1_rx_snr', 'an1_rx_powerdif', 'an1_tof']].copy()
df_an2 = df_tag[['tdoa21', 'snr_an2', 'power_dif_an2', 'an2_rx_snr', 'an2_rx_powerdif', 'an2_tof']].copy()

In [ ]:
# renaming cols
df_an1.rename({'tdoa12':'tdoa', 
                   'snr_an1':'snr_an', 
                   'power_dif_an1':'power_dif', 
                   'an1_rx_snr':'rx_snr', 
                   'an1_rx_powerdif':'rx_powerdif', 
                   'an1_tof':'tof'}, axis=1, inplace=True)

df_an2.rename({'tdoa21':'tdoa',
                   'snr_an2':'snr_an', 
                   'power_dif_an2':'power_dif', 
                   'an2_rx_snr':'rx_snr', 
                   'an2_rx_powerdif':'rx_powerdif', 
                   'an2_tof':'tof'}, axis=1, inplace=True)

In [ ]:
# adding information of nlos causing object
df_an1['NLOS_material'] = 1 # metal
df_an2['NLOS_material'] = 0 # nothing

In [ ]:
df = pd.concat([df_an1, df_an1], ignore_index=True, sort=False)
df.head()

In [ ]:
df.dropna(inplace=True)

## Classifying 

In [ ]:
# Import modules from Scikit-learn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split   # Import train_test_split function
from sklearn import metrics   # import metrics modules for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
# Create a pipeline object for the model
pipe_MLP = make_pipeline(StandardScaler(),
                         MLPClassifier(solver='adam',
                                       hidden_layer_sizes=(100,100,100,), # 3 hidden layers with (100x100x100) neurons
                                       random_state=0,
                                       max_iter=500,           # TODO: tune it later
                                       verbose=True
                                       )
                         )

In [ ]:
# Extracted features 
X = df[['tdoa', 'snr_an', 'power_dif', 'rx_snr', 'rx_powerdif', 'tof', 'NLOS_material']]
y = df['NLOS_material'] # Labels

In [ ]:
# Split dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test

In [ ]:
# Train the whole pipeline
pipe_MLP.fit(X_train, y_train)

In [ ]:
y_pred = pipe_MLP.predict(X_test)

In [ ]:
print(f'Test Accuracy: {metrics.accuracy_score(y_test, y_pred) * 100}%')